# Applied Data Science Capstone - Week 1 - Assignment

In [112]:
import pandas as pd
import numpy as np

In [113]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Applied Data Science Capstone - Week 3 - Assignment Segmenting and Clustering Neighborhoods in Toronto¶

In [114]:
!pip install geocoder
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geocoder # import geocoder
from geopy.geocoders import Nominatim

! pip install folium==0.5.0
import folium 
import json

Scrape Toronto postal table from the Wikipedia page and transform the data into a pandas dataframe

In [115]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",na_values=['No Acquirer'])
Neighbourhood=df[0]
Neighbourhood

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Drop boroughs that are Not assigned

In [116]:
Neighbourhood.drop(Neighbourhood[Neighbourhood['Borough']=='Not assigned'].index, inplace=True)

Groupby postal code and merge neighborhood

In [117]:
Neighbourhood.groupby('Postal Code').agg({'Borough':'sum','Neighbourhood':','.join})

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


Assign neighborhood to be the same as the borough if neighborhood Not assigned

In [118]:
Neighbourhood.loc[Neighbourhood['Neighbourhood']=='Not assigned','Neighbourhood'] = Neighbourhood.loc[Neighbourhood['Neighbourhood']=='Not assigned','Borough']

In [119]:
Neighbourhood

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Print the number of rows of your dataframe

In [120]:
Neighbourhood.shape

(103, 3)

Get latitude and longitude 

In [121]:
def get_latlon(postal_code):
# initialize your variable to None
    lat_lng_coords = None

# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

#column_names = ['Postal Code','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
latitudes=[]
longitudes=[]
for data in Neighbourhood['Postal Code']:
    lat_lng_coords=get_latlon(data)
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    latitudes.append(latitude)
    longitudes.append(longitude)
    
Neighbourhood['Latitude']=latitudes
Neighbourhood['Longitude']=longitudes
Neighbourhood

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.75245,-79.32991
3,M4A,North York,Victoria Village,43.73057,-79.31306
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
165,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


Creat table with only boroughs that contain the word Toronto

In [123]:
Toronto=Neighbourhood[Neighbourhood['Borough'].str.contains('Toronto')]
Toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
22,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
30,M4E,East Toronto,The Beaches,43.67709,-79.29547


Creat map of Toronto using latitude and longitude values

In [124]:
address = 'Toronto'

#geolocator = Nominatim(user_agent="ny_explorer")
#location = geocoder.arcgis(address)
latitude = 43.64840
longitude = -79.40180
map_Toronto=folium.Map(location=[latitude, longitude], zoom_start=11)

Add markers to map

In [125]:
for lat, lng, borough, postalcode in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Postal Code']):
    label = '{}, {}'.format(postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto